# Extração de Entidades Nomeadas - Pré-Processamento 

A extração de entidades nomeadas foi aplicada somente aos textos dos Depoentes e Convidados. Os textos estão no arquivo `input/discurso_dia_depoente_convidado.csv`. 

In [1]:
import pandas as pd 
import json

Esse documento foi gerado a partir da consulta: 

```sql
SELECT 
  data_sessao, 
  nome_discursante, 
  SUM(duracao_discurso) duracao_discursos, 
  STRING_AGG(texto_discurso, ' ') as discurso 
FROM 
  `basedosdados.br_senado_cpipandemia.discursos`
WHERE 
  categoria_discursante = 'Depoente/Convidado'
GROUP BY data_sessao, nome_discursante
ORDER BY data_sessao
```

In [17]:
all_data_depoente_convidado = pd.read_csv("../input/discurso_dia_depoente_convidado.csv")
all_data_depoente_convidado.head()

,data_sessao,nome_discursante,duracao_discursos,discurso
0,2021-05-04,LUIZ HENRIQUE MANDETTA,13680,Bom dia a todos! Prometo. Bom dia a todos e a ...
1,2021-05-05,NELSON LUIZ SPERLE TEICH,6960,Sim. Eu tinha escrito um texto de introdução. ...
2,2021-05-06,MARCELO QUEIROGA,8400,Posso fazer uma saudação aos Srs. Senadores. E...
3,2021-05-11,ANTONIO BARRA TORRES,9840,"Sim, senhor. Faço. Uma breve declaração, Sr. P..."
4,2021-05-12,FABIO WAJNGARTEN,7440,"Sim, sim. Eu gostaria, por favor, Sr. Presiden..."


Para a extração de entidades nomeadas foi utilizado o código disponibilizado neste repositório: https://github.com/neuralmind-ai/portuguese-bert. A execução é feita fora deste repositório. Mais detalhes podem se econtrados no repositório do projeto. 

Neste notebook são gerados os arquivos `.json` que servem de entrada para o algoritmo de extração de entidades. Esses arquivos estão na pasta `output/ner_depoentes_convidados/source`. Cada arquivo corresponde as falas de um Depoente ou Convidado de um dia específico da CPI. 

In [20]:
final_list_depoente_convidado = []

for index, row in all_data_depoente_convidado.iterrows():
    dict_ = {"doc_id": row['data_sessao'].replace("-","") + "_" + row['nome_discursante'].replace(" ","_"), "doc_text": row['discurso']}
    final_list_depoente_convidado.append(dict_)

In [26]:
for doc in final_list_depoente_convidado:
    file_path = "../output/ner_depoentes_convidados/source/" + doc["doc_id"] + ".json"
    with open(file_path, 'w') as outfile:
        json.dump([doc], outfile)

Cada arquivo foi processado pelo algoritmo de extração de entidades e gerado os arquivos da pasta `output/ner_depoentes_convidados/ner_files/`. Cada arquivo dessa pasta corresponde a um dia e um depoente/convidado da CPI. No arquivo é possível ver o texto processado e as entidades extraídas. Esses arquivos foram processados para gerar os arquivos finais. 